In [2]:
! pip install transformers datasets
! pip install evaluate
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.wh

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. Pre-training σε μεγάλα unlabelled datasets. Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. Fine-tuning σε μικρότερα labelled datasets. Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [ ]:
from datasets import load_dataset, concatenate_datasets

# insert your code here

# Download Yelp Polarity dataset
dataset = load_dataset("yelp_polarity")
dataset.shape



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for yelp_polarity contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/yelp_polarity
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.war

{'train': (560000, 2), 'test': (38000, 2)}

Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [ ]:
# insert your code here

# Filter and select 150 samples per class
train_dataset_0 = dataset['train'].filter(lambda e: e['label'] == 0).select(range(150))
train_dataset_1 = dataset['train'].filter(lambda e: e['label'] == 1).select(range(150))

test_dataset_0 = dataset['test'].filter(lambda e: e['label'] == 0).select(range(150))
test_dataset_1 = dataset['test'].filter(lambda e: e['label'] == 1).select(range(150))

# Concatenate the filtered datasets
train_dataset = concatenate_datasets([train_dataset_0, train_dataset_1]).shuffle()
test_dataset = concatenate_datasets([test_dataset_0, test_dataset_1]).shuffle()

# Print the first few examples to verify
print(train_dataset["label"][:10])
print(test_dataset["label"][:10])

[1, 1, 0, 1, 0, 0, 1, 0, 1, 0]
[1, 1, 1, 0, 1, 0, 0, 1, 1, 0]


In [ ]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

In [ ]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 300
})

# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [ ]:
# insert your code here
from transformers import AutoTokenizer, DistilBertForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print("Model architecture:", model.config.architectures[0])
print("Tokenizer:", tokenizer.__class__.__name__)

Model architecture: DistilBertForMaskedLM
Tokenizer: DistilBertTokenizerFast


Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# insert your code here

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 300
})

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [ ]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.

In [ ]:
!pip uninstall transformers accelerate -y
!pip install transformers[torch] accelerate -U


Found existing installation: transformers 4.41.0
Uninstalling transformers-4.41.0:
  Successfully uninstalled transformers-4.41.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 31.3 MB/s eta 0:00:00


In [ ]:
!pip cache purge

Files removed: 56


In [ ]:
#insert code here
from transformers import TrainingArguments, Trainer
from transformers import AdamW, Adafactor, get_scheduler

#args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16,num_train_epochs=3)

# Experiment with different hyperparameters
learning_rates = [3e-5, 1e-5]
batch_sizes = [8, 16, 32]
num_epochs = [3]
optimizers = {'AdamW': AdamW}
schedulers = {'linear': 'linear', 'cosine': 'cosine'}

for lr in learning_rates:
    for batch_size in batch_sizes:
        for epochs in num_epochs:
            for opt_name, opt_class in optimizers.items():
                for sched_name, sched_type in schedulers.items():
                    # Initialize the model from scratch
                    model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

                    # Set up training arguments
                    training_args = TrainingArguments(
                        output_dir="test_trainer",
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        num_train_epochs=epochs,
                        weight_decay=0.01,
                        learning_rate=lr,
                    )

                    # Initialize the optimizer
                    optimizer = opt_class(model.parameters(), lr=lr)

                    # Initialize the scheduler
                    num_training_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs
                    scheduler = get_scheduler(
                        name=sched_type, optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
                    )

                    trainer = Trainer(
                        model=model,
                        args=training_args,
                        train_dataset=train_dataset,
                        eval_dataset=test_dataset,
                        compute_metrics=compute_metrics,
                        optimizers=(optimizer, scheduler),
                    )

                    # Train the model
                    trainer.train()

                    # Evaluate and store the results
                    eval_result = trainer.evaluate()
                    accuracy = eval_result['eval_accuracy']
                    print(f"LR: {lr}, Batch Size: {batch_size}, Epochs: {epochs}, Optimizer: {opt_name}, Scheduler: {sched_name}, Accuracy: {accuracy}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.402907,0.873333
2,No log,0.246183,0.913333
3,No log,0.249662,0.910000


LR: 3e-05, Batch Size: 8, Epochs: 3, Optimizer: AdamW, Scheduler: linear, Accuracy: 0.91


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.455855,0.853333
2,No log,0.272594,0.900000
3,No log,0.267533,0.900000


...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.689249,0.563333
2,No log,0.683313,0.676667
3,No log,0.681747,0.700000


LR: 1e-05, Batch Size: 32, Epochs: 3, Optimizer: AdamW, Scheduler: linear, Accuracy: 0.7


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.688598,0.556667
2,No log,0.682852,0.683333
3,No log,0.682132,0.690000


LR: 1e-05, Batch Size: 32, Epochs: 3, Optimizer: AdamW, Scheduler: cosine, Accuracy: 0.69


Έχουμε τα παρακάτω αποτελέσματα:

LR: 3e-05, Batch Size: 8, Epochs: 3, Optimizer: AdamW, Scheduler: linear, Accuracy: 0.91

LR: 3e-05, Batch Size: 8, Epochs: 3, Optimizer: AdamW, Scheduler: cosine, Accuracy: 0.9

LR: 3e-05, Batch Size: 16, Epochs: 3, Optimizer: AdamW, Scheduler: linear, Accuracy: 0.8333333333333334

LR: 3e-05, Batch Size: 16, Epochs: 3, Optimizer: AdamW, Scheduler: cosine, Accuracy: 0.8366666666666667

LR: 3e-05, Batch Size: 32, Epochs: 3, Optimizer: AdamW, Scheduler: linear, Accuracy: 0.7333333333333333

LR: 3e-05, Batch Size: 32, Epochs: 3, Optimizer: AdamW, Scheduler: cosine, Accuracy: 0.74

LR: 1e-05, Batch Size: 8, Epochs: 3, Optimizer: AdamW, Scheduler: linear, Accuracy: 0.81

LR: 1e-05, Batch Size: 8, Epochs: 3, Optimizer: AdamW, Scheduler: cosine, Accuracy: 0.7766666666666666

LR: 1e-05, Batch Size: 16, Epochs: 3, Optimizer: AdamW, Scheduler: linear, Accuracy: 0.7366666666666667

LR: 1e-05, Batch Size: 16, Epochs: 3, Optimizer: AdamW, Scheduler: cosine, Accuracy: 0.73

LR: 1e-05, Batch Size: 32, Epochs: 3, Optimizer: AdamW, Scheduler: linear, Accuracy: 0.7

LR: 1e-05, Batch Size: 32, Epochs: 3, Optimizer: AdamW, Scheduler: cosine, Accuracy: 0.69




**Best Hyperparameters:** For this specific model and task, the best performance is achieved with a higher learning rate (3e-05), smaller batch size (8), and linear scheduler. This combination maximizes the accuracy of the fine-tuned model.

**Trends Observed:** Lower learning rates and larger batch sizes generally lead to lower accuracy, likely due to insufficient model adjustments during training and overly smoothed gradients.

**Optimization Strategy:** When fine-tuning a model, starting with a higher learning rate and smaller batch size while experimenting with schedulers can yield better results. Further fine-tuning can then involve adjusting these parameters based on observed performance to find the optimal balance for the specific task and dataset.

In [ ]:
args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", per_device_train_batch_size=16, learning_rate=0.0005, num_train_epochs=3, optim= "adamw_torch")

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trained_model=trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.721075,0.500000
2,No log,0.709391,0.500000
3,No log,0.693362,0.500000


# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
# # insert your code here (load dataset)
from datasets import load_dataset
ds = load_dataset('piqa')
ds

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for piqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/piqa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 16113
    })
    test: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 3084
    })
    validation: Dataset({
        features: ['goal', 'sol1', 'sol2', 'label'],
        num_rows: 1838
    })
})

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [ ]:
val_100 = ds['validation'].shuffle().select(range(100))
val_100

Dataset({
    features: ['goal', 'sol1', 'sol2', 'label'],
    num_rows: 100
})

In [ ]:
# insert your code here (models)
from transformers import AutoTokenizer, AutoModelForMultipleChoice

tokenizer1 = AutoTokenizer.from_pretrained("bert-base-uncased")
model1 = AutoModelForMultipleChoice.from_pretrained("bert-base-uncased")

tokenizer2 = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model2 = AutoModelForMultipleChoice.from_pretrained("distilbert-base-uncased")

tokenizer3 = AutoTokenizer.from_pretrained("roberta-base")
model3 = AutoModelForMultipleChoice.from_pretrained("roberta-base")

tokenizer4 = AutoTokenizer.from_pretrained("albert-base-v2")
model4 = AutoModelForMultipleChoice.from_pretrained("albert-base-v2")

tokenizer5 = AutoTokenizer.from_pretrained("google/electra-base-discriminator")
model5 = AutoModelForMultipleChoice.from_pretrained("google/electra-base-discriminator")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of ElectraForMultipleChoice were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# insert your code here (function for ending prediction)

# For each model tokenizer we define a tokenize_function that is applied to the preprocessed dataset and we take 5 different tokenized sets
def preprocess(example):
    example['first_sentence']=[example['goal']]*2
    example['second_sentence']=[example[f'sol{i}'] for i in [1,2]]
    return example

def tokenize_function1(examples):
    tokenized_examples=tokenizer1(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples

def tokenize_function2(examples):
    tokenized_examples=tokenizer2(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples

def tokenize_function3(examples):
    tokenized_examples=tokenizer3(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples

def tokenize_function4(examples):
    tokenized_examples=tokenizer4(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True)
    return tokenized_examples

def tokenize_function5(examples):
    return(tokenizer5(examples['first_sentence'],examples['second_sentence'],padding='max_length',truncation=True))

In [ ]:
val_100_processed = val_100.map(preprocess)
tokenized1_val_100 = val_100_processed.map(tokenize_function1)
tokenized2_val_100 = val_100_processed.map(tokenize_function2)
tokenized3_val_100 = val_100_processed.map(tokenize_function3)
tokenized4_val_100 = val_100_processed.map(tokenize_function4)
tokenized5_val_100 = val_100_processed.map(tokenize_function5)


In [ ]:
# Each tokenizer uses a different tokenized set and the same compute metric with the others. By calling trainer.evaluate we do not have to define
# an accuracy computing function ourselves
trainer = Trainer(
    model=model1,
    eval_dataset=tokenized1_val_100,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.6898735165596008,
 'eval_accuracy': 0.58,
 'eval_runtime': 6.0692,
 'eval_samples_per_second': 16.477,
 'eval_steps_per_second': 2.142}

In [ ]:
trainer = Trainer(
    model=model2,
    eval_dataset=tokenized2_val_100,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.692908763885498,
 'eval_accuracy': 0.5,
 'eval_runtime': 3.3746,
 'eval_samples_per_second': 29.633,
 'eval_steps_per_second': 3.852}

In [ ]:
trainer = Trainer(
    model=model3,
    eval_dataset=tokenized3_val_100,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.6931206583976746,
 'eval_accuracy': 0.47,
 'eval_runtime': 5.9314,
 'eval_samples_per_second': 16.859,
 'eval_steps_per_second': 2.192}

In [ ]:
trainer = Trainer(
    model=model4,
    eval_dataset=tokenized4_val_100,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.6890628337860107,
 'eval_accuracy': 0.59,
 'eval_runtime': 7.5074,
 'eval_samples_per_second': 13.32,
 'eval_steps_per_second': 1.732}

In [ ]:
trainer = Trainer(
    model=model5,
    eval_dataset=tokenized5_val_100,
    compute_metrics=compute_metrics,
)
trainer.evaluate()

{'eval_loss': 0.6927877068519592,
 'eval_accuracy': 0.58,
 'eval_runtime': 5.9338,
 'eval_samples_per_second': 16.853,
 'eval_steps_per_second': 2.191}

## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.4048847

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [ ]:
# insert your code here (load dataset)
from datasets import load_dataset
ds = load_dataset('truthful_qa','generation')

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

In [ ]:
# We ensure that each sample of the dataset has more than 2 correct answers
ds_filtered = ds['validation'].filter(lambda e: len(e['correct_answers'])>=2).shuffle().select(range(100))

ds_filtered_with_label = ds_filtered.add_column('label', [0]*100)
ds_filtered_with_label

Filter:   0%|          | 0/817 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['type', 'category', 'question', 'best_answer', 'correct_answers', 'incorrect_answers', 'source', 'label'],
    num_rows: 100
})

In [ ]:
# insert your code here (load models for semantic similarity and QA)

from sentence_transformers import SentenceTransformer

sentence_model1 = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
sentence_model2 = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
sentence_model3 = SentenceTransformer('shibing624/text2vec-base-chinese')
sentence_model4 = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
sentence_model5 = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
sentence_model6 = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')


def preprocess2(example):
    example['first_sentence']=[example['question']]*4
    example['second_sentence']=[example['best_answer'],example['correct_answers'][0],example['correct_answers'][1],example['incorrect_answers'][0]]
    return example


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
ds_filtered_with_label_processed = ds_filtered_with_label.map(preprocess2)

tokenized1_val_100 = ds_filtered_with_label_processed.map(tokenize_function1)
tokenized2_val_100 = ds_filtered_with_label_processed.map(tokenize_function2)
tokenized3_val_100 = ds_filtered_with_label_processed.map(tokenize_function3)
tokenized4_val_100 = ds_filtered_with_label_processed.map(tokenize_function4)
tokenized5_val_100 = ds_filtered_with_label_processed.map(tokenize_function5)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
import evaluate
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics2(eval_pred, sentence_model):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    optpredictions = optimal_predictions(predictions, sentence_model)
    return metric.compute(predictions=optpredictions, references=labels)
# We define a second accuracy computing function for calculating the percentage of optimal correct answers

In [ ]:
# insert your code here (function for optimal correct answers & semantic similarity)

def optimal_predictions(predictions,model):
    for i in range(len(predictions)):
      if (predictions[i]==0):
        predictions[i]=0
      elif (predictions[i]==3):
        predictions[i]=1
      else :
        sentences = [ds_filtered_with_label[i]['correct_answers'][predictions[i]-1], ds_filtered_with_label[i]['best_answer']]
        embeddings = model.encode(sentences)
        if (get_cosine_similarity(embeddings[0], embeddings[1])>0.95):
          predictions[i]=0
        else:
          predictions[i]=1
    return predictions

# If the model predict a correct answer instead of a best answer, this counts a correct only if the correct answer has a sentence similarity
# of more than 0.95. So we try all models and all sentence models to this aim and we note all the corresponding model accuracies for optimal correctness

In [ ]:
# Define a list of models and datasets

models_and_datasets = [
    (model1, tokenized1_val_100, "bert-base-uncased"),
    (model2, tokenized2_val_100, "distilbert-base-uncased"),
    (model3, tokenized3_val_100, "roberta-base"),
    (model4, tokenized4_val_100, "albert-base-v2"),
    (model5, tokenized5_val_100, "google/electra-base-discriminator"),
]

# Sentence Transformer models
sentence_models = [
    ("sentence-transformers/all-mpnet-base-v2", sentence_model1),
    ("sentence-transformers/all-MiniLM-L6-v2", sentence_model2),
    ("shibing624/text2vec-base-chinese", sentence_model3),
    ("sentence-transformers/multi-qa-mpnet-base-dot-v1", sentence_model4),
    ("sentence-transformers/all-MiniLM-L12-v2", sentence_model5),
    ("sentence-transformers/bert-base-nli-mean-tokens", sentence_model6)
]


In [ ]:
import numpy as np
import evaluate
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics2(eval_pred, sentence_model):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    optpredictions = optimal_predictions(predictions, sentence_model)
    return metric.compute(predictions=optpredictions, references=labels)
# We define a second accuracy computing function for calculating the percentage of optimal correct answers

In [ ]:
from transformers import Trainer
metric = evaluate.load("accuracy")

results = []

# Evaluate the multiple-choice models with sentence transformers
for model, tokenized_val_dataset, model_name in models_and_datasets:
    for sentence_model_name, sentence_model in sentence_models:
        # Initialize Trainer
        trainer = Trainer(
            model=model,
            eval_dataset=tokenized_val_dataset,
            compute_metrics=lambda eval_pred: compute_metrics2(eval_pred, sentence_model),
        )

        # Evaluate the model
        eval_result = trainer.evaluate()
        accuracy = eval_result["eval_accuracy"]

        # Store the results
        results.append((model_name, sentence_model_name, accuracy))

In [ ]:
# Print results as a markdown table using tabulate
from tabulate import tabulate
headers = ["Model", "Sentence Model", "Accuracy"]
table = tabulate(results, headers=headers, tablefmt="github")
print(table)

| Model                             | Sentence Model                                   |   Accuracy |
|-----------------------------------|--------------------------------------------------|------------|
| bert-base-uncased                 | sentence-transformers/all-mpnet-base-v2          |       0.37 |
| bert-base-uncased                 | sentence-transformers/all-MiniLM-L6-v2           |       0.35 |
| bert-base-uncased                 | shibing624/text2vec-base-chinese                 |       0.35 |
| bert-base-uncased                 | sentence-transformers/multi-qa-mpnet-base-dot-v1 |       0.36 |
| bert-base-uncased                 | sentence-transformers/all-MiniLM-L12-v2          |       0.35 |
| bert-base-uncased                 | sentence-transformers/bert-base-nli-mean-tokens  |       0.39 |
| distilbert-base-uncased           | sentence-transformers/all-mpnet-base-v2          |       0.34 |
| distilbert-base-uncased           | sentence-transformers/all-MiniLM-L6-v2      

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [3]:
# insert your code here (load dataset)
from datasets import load_dataset
ds = load_dataset('winogrande','winogrande_debiased')
ds

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for winogrande contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/winogrande
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/9248 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 9248
    })
    test: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 1767
    })
    validation: Dataset({
        features: ['sentence', 'option1', 'option2', 'answer'],
        num_rows: 1267
    })
})

In [4]:
val_100 = ds['validation'].shuffle(seed=42).select(range(100))

Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [5]:
# insert your code here (load models)
from transformers import pipeline

# insert your code here (create pipelines)
classifier1 = pipeline(model="DeepPavlov/roberta-large-winogrande")
classifier2 = pipeline(model="sileod/deberta-v3-base-tasksource-nli")
classifier3 = pipeline(model="sileod/deberta-v3-large-tasksource-nli")

config.json:   0%|          | 0.00/820 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

We define an accuracy calculating function for each of the 3 models. This is necessary as each model returns different labels for classifying a training dataset and we can infer from these labels the correct label for our specific problem. As there is not a pipeline for multiple choice task, this task was reduced to  sequence classification for the first two models and zero-shot classification for the third (the labels in the third model are inserted by the user)

In [10]:
# insert your code here (function for predicting best fill)
# classifier1
acc=0
for i in range(100):
  sentence_1,sentence_2 = str(val_100[i]['sentence']).split('_')
  sentence_2a = str(val_100[i]['option1']) + sentence_2
  sentence_2b = str(val_100[i]['option2']) + sentence_2
  sentence_1,sentence_2 = sentence_1 + sentence_2a, sentence_1 + sentence_2b
  class1,class2 = classifier1(sentence_1)[0], classifier1(sentence_2)[0]
  label1,label2 = class1['label'], class2['label']
  score1,score2 = class1['score'], class2['score']
  if (label1 != label2):
    if (label1=='True'):
      if (val_100[i]['answer']) == '1':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '2':
        acc=acc+1
  elif(label1=='True'):
    if (score1>score2):
      if (val_100[i]['answer']) == '1':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '2':
        acc=acc+1
  else:
    if (score1>score2):
      if (val_100[i]['answer']) == '2':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '1':
        acc=acc+1

print(f"Accuracy: {acc/100.0:.2f}")


Accuracy: 0.68


In [9]:
# Define the labels for entailment
labels = ["entailment", "neutral", "contradiction"]

# Accuracy calculation
acc = 0
for i in range(100):
    # Extract sentences and options
    sentence_1, sentence_2 = str(val_100[i]['sentence']).split('_')
    sentence_2a = str(val_100[i]['option1']) + sentence_2
    sentence_2b = str(val_100[i]['option2']) + sentence_2
    full_sentence_1 = sentence_1 + sentence_2a
    full_sentence_2 = sentence_1 + sentence_2b

    # Get classifications and scores
    class1 = classifier2(full_sentence_1, candidate_labels=labels)
    class2 = classifier2(full_sentence_2, candidate_labels=labels)

    # Extract scores for entailment
    for i in range(3):
        if class1['labels'][i] == 'entailment':
            score1 = class1['scores'][i]
        if class2['labels'][i] == 'entailment':
            score2 = class2['scores'][i]

    # Determine the predicted option based on higher score
    if (score1 > score2 and val_100[i]['answer'] == '1'):
        acc += 1
    if (score1 < score2 and val_100[i]['answer'] == '2'):
        acc += 1

# Calculate and print accuracy
print(f"Accuracy: {acc / 100.0:.2f}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Accuracy: 0.50


In [ ]:
# insert your code here (function for predicting best fill)
# classifier3
acc=0
for i in range(100):
  sentence_1,sentence_2 = str(val_100[i]['sentence']).split('_')
  sentence_2a = str(val_100[i]['option1']) + sentence_2
  sentence_2b = str(val_100[i]['option2']) + sentence_2
  sentence_1,sentence_2 = sentence_1 + sentence_2a, sentence_1 + sentence_2b
  class1,class2 = classifier3(sentence_1,['True','False']), classifier3(sentence_2,['True','False'])
  label1,label2 = class1['labels'][0], class2['labels'][0]
  score1,score2 = class1['scores'][0], class2['scores'][0]
  if (label1 != label2):
    if (label1=='True'):
      if (val_100[i]['answer']) == '1':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '2':
        acc=acc+1
  elif(label1=='True'):
    if (score1>score2):
      if (val_100[i]['answer']) == '1':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '2':
        acc=acc+1
  else:
    if (score1>score2):
      if (val_100[i]['answer']) == '2':
        acc=acc+1
    else:
      if (val_100[i]['answer']) == '1':
        acc=acc+1

print(f"Accuracy: {acc/100.0:.2f}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Accuracy: 0.81
